# 데이터
---
- 데이터는 QnA데이터로 총 11,823의 대화가 있습니다.
- label은 0 [일상], 1 [부정], 2 [긍정] 이 있습니다.
- 총 데이터 크기는 약 869KB 입니다.

데이터 예시는 다음과 같습니다.

In [1]:
import pandas as pd

data = pd.read_csv(r"C:\Users\loveg\Downloads\Chatbot_data-master\ChatbotData.csv")

In [2]:
data

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0
...,...,...,...
11818,훔쳐보는 것도 눈치 보임.,티가 나니까 눈치가 보이는 거죠!,2
11819,훔쳐보는 것도 눈치 보임.,훔쳐보는 거 티나나봐요.,2
11820,흑기사 해주는 짝남.,설렜겠어요.,2
11821,힘든 연애 좋은 연애라는게 무슨 차이일까?,잘 헤어질 수 있는 사이 여부인 거 같아요.,2


# 데이터 로더

- 위의 data를 불러올 데이터 로더 클래스를 선언 합니다.
- 데이터를 Q와 A를 이어줍니다.

In [3]:
import csv
from torch.utils.data import Dataset, DataLoader


class ConversationDataset(Dataset):
    def __init__(self, data_path):
        self.conversation_data = []
        self.end_of_text_token = "</s>"

        header = 0
        with open(data_path, encoding='utf-8') as csv_file:
            csv_reader = csv.reader(csv_file, delimiter=',')

            for row in csv_reader:
                if header == 0:
                    header += 1
                    continue
                _type = ["일상", "부정", "긍정"][int(row[2])]
                temp_converation = [f"{_type}: {row[0]}{self.end_of_text_token}", f"{row[1]}{self.end_of_text_token}"]
                self.conversation_data.append(temp_converation)

    def __len__(self):
        return len(self.conversation_data)

    def __getitem__(self, idx):
        return self.conversation_data[idx]

## 디바이스 설정

In [4]:
import torch

device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'

## 파라미터 설정

- BATCH_SIZE : 데이터 로더의 배치 사이즈를 설정합니다.
- EPOCHS : Fine-Tuning할 Epoch을 설정합니다.
- LEARNING_RATE : 모델 학습시, lr을 설정합니다.
- WARMUP_STEPS : 스케쥴러의 warmup을 진행할 step을 설정합니다.
- DATA_PATH : Fine-Tuning에 사용할 데이터 경로를 지정합니다.
- MODEL_TYPE : 허깅페이스의 KoGPT-2를 설정합니다.
- OUTPUT_FOLDER : 모델 저장 경로를 지정해 줍니다.

In [5]:
BATCH_SIZE = 1
EPOCHS = 3
LEARNING_RATE = 3e-5
WARMUP_STEPS = 100
MAX_SEQ_LEN = 100
DATA_PATH = r"C:\Users\loveg\Downloads\Chatbot_data-master\ChatbotData.csv"
MODEL_TYPE = "taeminlee/kogpt2"
OUTPUT_FOLDER = r"E:\online_task"

## Pre-trained MODEL LOAD
- 허깅페이스의 KoGPT2모델 및 tokenizer를 Load합니다.

In [6]:
from transformers import GPT2LMHeadModel, PreTrainedTokenizerFast


model = GPT2LMHeadModel.from_pretrained(MODEL_TYPE)
tokenizer = PreTrainedTokenizerFast.from_pretrained(MODEL_TYPE)

## dataset 생성 및 dataloader 생성

In [7]:
dataset = ConversationDataset(data_path=DATA_PATH)
data_loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

In [8]:
for d in data_loader:
    print(d)
    break

[('부정: 와 진짜 너무 짜증나네</s>',), ('제가 도움이 되고 싶네요.</s>',)]


## Fine-Tune Phase

In [9]:
from transformers import AdamW, get_linear_schedule_with_warmup


# 모델을 GPU로 보냅니다.
model = model.to(device)

# 모델을 학습하기위해 Optimizer와 scheduler를 선언 합니다.
model.train()
optimizier = AdamW(model.parameters(), lr=LEARNING_RATE)
scheduler = get_linear_schedule_with_warmup(optimizier, WARMUP_STEPS, len(data_loader) - WARMUP_STEPS, -1)

모델 저장할 곳이 없을 경우 생성합니다.

In [10]:
import os


if not os.path.exists(OUTPUT_FOLDER):
    os.mkdir(OUTPUT_FOLDER)

start_token과 unknown_token을 가져옵니다.

In [11]:
start_token = torch.tensor(tokenizer.encode("<s>")).unsqueeze(0)
unknown_token = tokenizer.unk_token_id

모델 FineTuning 코드입니다.
(전처리 단계에서 미리 Q와 A를 합쳐두는 것이 좋을 것 같습니다.)


우선, 해당 코드를 실행했습니다.

In [12]:
from tqdm.notebook import tqdm


for epoch in range(EPOCHS):
    print(f'EPOCH : {epoch}, started' + "=" * 30)
    proc_seq_count = 0
    total_loss = 0.0
    total_count = 0
    temp_count = 0  # 노트북에서 보여드리기 위한 변수입니다.
    # 실제 실행시, temp_count와 관련된 코드들은 삭제해주세요.
    # 여기 부분
    with tqdm(data_loader, desc="Train Epoch #{}".format(epoch)) as t:
        for idx, data in enumerate(t):
            if temp_count > 5: # 여기
                break          # 여기
            temp_count += 1    # 여기
            Q, A = tokenizer.encode(data[0][0]), tokenizer.encode(data[1][0])
            for i in range(len(Q)):
                if Q[i] > tokenizer.vocab_size:
                    Q[i] = unknown_token
            for i in range(len(A)):
                if A[i] > tokenizer.vocab_size:
                    A[i] = unknown_token

            Q = torch.tensor(Q).unsqueeze(0)
            A = torch.tensor(A).unsqueeze(0)
            temp_conversation = torch.cat([Q, start_token, A], dim=1).to(device)

            outputs = model(temp_conversation, labels=temp_conversation)
            loss, logits = outputs[:2]
            loss.backward()
            total_loss += loss.detach().data
            total_count += 1
            t.set_postfix(loss='{:.6f}'.format(total_loss / total_count))

            proc_seq_count += 1
            if proc_seq_count == BATCH_SIZE:
                proc_seq_count = 0
                optimizier.step()
                scheduler.step()
                optimizier.zero_grad()
                scheduler.optimizer.zero_grad()

            if idx % 2000 == 1:
                torch.save(model.state_dict(), os.path.join(OUTPUT_FOLDER, f"KoGPT2_KoDialog_{epoch}_{idx}.pt"))
        torch.save(model.state_dict(), os.path.join(OUTPUT_FOLDER, f"KoGPT2_KoDialog_{epoch}.pt"))

EPOCH : 0, started==============================



EPOCH : 1, started==============================



EPOCH : 2, started==============================


## Sentence-Generate
- FINETUNE_MODEL_PATH : 학습된 모델의 가중치 경로를 설정합니다.

In [13]:
FINETUNE_MODEL_PATH = r"E:\online_task\KoGPT2_KoDialog_3.pt"

QnA_Service_MODEL에 해당 가중치를 불러와 load 시켜 줍니다.

In [14]:
QnA_Service_MODEL = GPT2LMHeadModel.from_pretrained("taeminlee/kogpt2")
QnA_Service_MODEL.load_state_dict(torch.load(FINETUNE_MODEL_PATH))

<All keys matched successfully>

In [15]:
example_text = "3박4일 정도 놀러가고 싶다"
encoded_text = tokenizer.encode(example_text, add_special_tokens=True, return_tensors="pt")

In [16]:
encoded_text

tensor([[  141, 47650, 47514, 47471,  1057,  2211, 47593,  2999,  5314]])

In [17]:
generated_sentence = QnA_Service_MODEL.generate(encoded_text)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
C:\Users\loveg\Anaconda3\lib\site-packages\transformers\generation_utils.py:963: UserWarning: `max_length` is deprecated in this function, use `stopping_criteria=StoppingCriteriaList(MaxLengthCriteria(max_length=max_length))` instead.
  warnings.warn(


In [18]:
generated_sentence

tensor([[  141, 47650, 47514, 47471,  1057,  2211, 47593,  2999,  5314, 47440,
             1,     0, 18702,  8274, 27043, 47440,     1,     0, 18702,  8274]])

In [19]:
decoded_text = tokenizer.decode(generated_sentence[0], skip_special_tokens=True)

In [20]:
decoded_text

'3박4일 정도 놀러가고 싶다. 저도 가고 싶어요. 저도 가고'